# Web Scraping Practice

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

##### Define GetSoup function for future use

In [3]:
def getSoup(url):
  try:
    resp=requests.get(url)
    resp.encoding='utf-8'
    if resp.status_code == 200:
      return BeautifulSoup(resp.text,'lxml')
    else:
      return 'Error:Status_code'+ str(resp.status_code)

  except Exception as e:
    return e

### Cathay United Bank Currency Exchange Rate

In [5]:
url='https://www.cathaybk.com.tw/cathaybk/personal/deposit-exchange/rate/currency-billboard/'
ResPage = getSoup(url)

In [ ]:
CurrencyTable = ResPage.find_all('div', class_='cubre-o-table');
len(CurrencyTable) 

##### Get Column and Data Row List

> RWD Design, Use CurrencyTable[0] as main data source 

In [ ]:
List_CurrencyTable = CurrencyTable[0].find_all('div', class_='cubre-o-table__item currency'); List_CurrencyTable

> Use first section in List_CurrencyTable to build up data structure

>1.  Find currency name

In [19]:
Currency_Name = List_CurrencyTable[0].find('div', class_='cubre-m-currency__name')
Currency_Name = Currency_Name.text.replace('\r\n','').strip()
Currency_Name

'美元USD'

>2. Find the value of exchange rate

In [ ]:
SubCurrencyTable = List_CurrencyTable[0].find('table', class_='cubre-m-rateTable -currency')
SubCurrencyTable

In [42]:
TempList = SubCurrencyTable.find_all('div', class_='cubre-m-rateTable__th');TempList
TitleList = [ item.text for item in TempList if TempList.index(item)%2 == 0 ] #取偶數項目
TitleList

['項目', '銀行買進', '銀行賣出']

In [ ]:
TempList1 = SubCurrencyTable.find('tbody').find_all('tr');TempList1

In [55]:
TempList2 = TempList1[0].find_all('div');TempList2

[<div class="cubre-m-rateTable__name">即期匯率</div>,
 <div>31.3500</div>,
 <div>31.4700</div>]

In [58]:
ColumnsList = []

ColumnsList.append(f"{TempList2[0].text}({TitleList[1]})")
ColumnsList.append(f"{TempList2[0].text}({TitleList[2]})")
ColumnsList

['即期匯率(銀行買進)', '即期匯率(銀行賣出)']

In [59]:
RowList = []

RowList.append(f"{TempList2[1].text}")
RowList.append(f"{TempList2[2].text}")

RowList

['31.3500', '31.4700']

In [75]:
ColumnsList = ['Currency']
RowList = [Currency_Name]
intCounter = 1

for LoopList1 in SubCurrencyTable.find('tbody').find_all('tr'):
    intCounter = 1
    for LoopList2 in LoopList1.find_all('div'):
        # print(LoopList2.text)
        if(intCounter==1):
            ColumnsList.append(f"{LoopList2.text}({TitleList[1]})")
            ColumnsList.append(f"{LoopList2.text}({TitleList[2]})")
        elif(intCounter==2):
            RowList.append(f"{LoopList2.text}")
        else:
            RowList.append(f"{LoopList2.text}")
        intCounter += 1


In [76]:
ColumnsList

['Currency',
 '即期匯率(銀行買進)',
 '即期匯率(銀行賣出)',
 '數位通路優惠匯率(銀行買進)',
 '數位通路優惠匯率(銀行賣出)',
 '現鈔匯率(銀行買進)',
 '現鈔匯率(銀行賣出)']

In [77]:
RowList

['美元USD', '31.3500', '31.4700', '31.3800', '31.4400', '31.0700', '31.6900']

##### Traversal the entire table

In [83]:
ColumnsList = ['Currency']
Datas = []
RowList = []
intCounter = 1

List_CurrencyTable = CurrencyTable[0].find_all('div', class_='cubre-o-table__item currency'); List_CurrencyTable

for TempTable in CurrencyTable[0].find_all('div', class_='cubre-o-table__item currency'):

    #Get currency_Name and append to row list
    TempCountry_Name = TempTable.find('div', class_='cubre-m-currency__name').text.replace('\r\n','').strip()
    # TempCountry_Name = TempCountry_Name
    RowList.append(TempCountry_Name)

    #Get Heads and Exchange Rate
    ColumnsList = ['Currency']
    SubCurrencyTable = TempTable.find('table', class_='cubre-m-rateTable -currency')
    for LoopList1 in SubCurrencyTable.find('tbody').find_all('tr'):
        intCounter = 1
        
        for LoopList2 in LoopList1.find_all('div'):
            
            if(intCounter==1):
                ColumnsList.append(f"{LoopList2.text}({TitleList[1]})")
                ColumnsList.append(f"{LoopList2.text}({TitleList[2]})")
            elif(intCounter==2):
                RowList.append(f"{LoopList2.text}")
            else:
                RowList.append(f"{LoopList2.text}")
            intCounter += 1
    
    Datas.append(RowList)
    RowList = []


In [81]:
ColumnsList

['Currency',
 '即期匯率(銀行買進)',
 '即期匯率(銀行賣出)',
 '數位通路優惠匯率(銀行買進)',
 '數位通路優惠匯率(銀行賣出)',
 '現鈔匯率(銀行買進)',
 '現鈔匯率(銀行賣出)']

In [84]:
Datas

[['美元USD', '31.3500', '31.4700', '31.3800', '31.4400', '31.0700', '31.6900'],
 ['歐元EUR', '33.9200', '34.4200', '33.9800', '34.3600', '33.4900', '34.7700'],
 ['日圓JPY', '0.2116', '0.2166', '0.2126', '0.2156', '0.2076', '0.2186'],
 ['人民幣CNY', '4.3140', '4.3890', '4.3240', '4.3790', '4.2140', '4.4090'],
 ['港幣HKD', '3.9790', '4.0490', '3.9890', '4.0390', '3.8990', '4.0890'],
 ['澳大利亞幣AUD', '20.5900', '20.8500', '20.6200', '20.8200', '--', '--'],
 ['紐西蘭幣NZD', '19.1800', '19.4800', '19.2000', '19.4600', '--', '--'],
 ['南非幣ZAR', '1.6010', '1.7210', '1.6060', '1.7160', '--', '--'],
 ['加拿大幣CAD', '23.1000', '23.5000', '23.1300', '23.4700', '--', '--'],
 ['英鎊GBP', '39.4400', '40.0600', '39.5000', '40.0000', '--', '--'],
 ['瑞士法郎CHF', '36.2700', '36.7900', '36.3000', '36.7600', '--', '--'],
 ['瑞典幣SEK', '2.9540', '3.0740', '2.9640', '3.0640', '--', '--'],
 ['新加坡幣SGD', '23.2700', '23.5700', '23.3000', '23.5400', '--', '--'],
 ['泰國幣THB', '0.8580', '0.9180', '0.8610', '0.9150', '--', '--'],
 ['丹麥幣DKK', '

##### Export Data as Excel File

In [85]:
df = pd.DataFrame(Datas,columns=ColumnsList)
df

,Currency,即期匯率(銀行買進),即期匯率(銀行賣出),數位通路優惠匯率(銀行買進),數位通路優惠匯率(銀行賣出),現鈔匯率(銀行買進),現鈔匯率(銀行賣出)
0,美元USD,31.3500,31.4700,31.3800,31.4400,31.0700,31.6900
1,歐元EUR,33.9200,34.4200,33.9800,34.3600,33.4900,34.7700
2,日圓JPY,0.2116,0.2166,0.2126,0.2156,0.2076,0.2186
3,人民幣CNY,4.3140,4.3890,4.3240,4.3790,4.2140,4.4090
4,港幣HKD,3.9790,4.0490,3.9890,4.0390,3.8990,4.0890
5,澳大利亞幣AUD,20.5900,20.8500,20.6200,20.8200,--,--
6,紐西蘭幣NZD,19.1800,19.4800,19.2000,19.4600,--,--
7,南非幣ZAR,1.6010,1.7210,1.6060,1.7160,--,--
8,加拿大幣CAD,23.1000,23.5000,23.1300,23.4700,--,--
9,英鎊GBP,39.4400,40.0600,39.5000,40.0000,--,--


In [86]:
df.to_csv('./Output/Quiz_2/CathayUnitedBank_CurrencyExchangeRate.csv', encoding='utf-8-sig')